In [75]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import numpy as np
import xgboost as xgb

In [76]:
X = np.load("./tatanic_X_train.npy")
y = np.load("./tatanic_y_train.npy")

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3)

In [82]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=1000, max_depth=2, learning_rate=0.5, nthread=7)  #라운드 1000번 max_depth를 짧게 variance낮은 모델 #cost function을 떨어뜨리는 learning_rate
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [83]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.7752808988764045

In [84]:
dtrain = xgb.DMatrix(X_train, label=y_train)  #여기서부터 학습과정 보여주는 코드 
dtest = xgb.DMatrix(X_test, label=y_test)

In [85]:
param = {'max_depth': 2, 'eta': 0.5, 'silent': 1, 'objective': 'binary:logistic'} #파라미터 조정
param['nthread'] = 7
param['eval_metric'] = 'auc'
evallist = [(dtest, 'eval'), (dtrain, 'train')]
plst = param.items()

In [89]:
num_round = 50
bst = xgb.train(plst, dtrain, num_round, evallist)  #여기까지

[0]	eval-auc:0.791547	train-auc:0.835036
[1]	eval-auc:0.827175	train-auc:0.865661
[2]	eval-auc:0.84269	train-auc:0.880466
[3]	eval-auc:0.850391	train-auc:0.881838
[4]	eval-auc:0.855074	train-auc:0.885557
[5]	eval-auc:0.85335	train-auc:0.888847
[6]	eval-auc:0.859125	train-auc:0.893359
[7]	eval-auc:0.856798	train-auc:0.897279
[8]	eval-auc:0.859355	train-auc:0.89913
[9]	eval-auc:0.859413	train-auc:0.898238
[10]	eval-auc:0.861481	train-auc:0.901428
[11]	eval-auc:0.863119	train-auc:0.906084
[12]	eval-auc:0.86513	train-auc:0.908833
[13]	eval-auc:0.865791	train-auc:0.910166
[14]	eval-auc:0.853005	train-auc:0.914052
[15]	eval-auc:0.851281	train-auc:0.916919
[16]	eval-auc:0.852086	train-auc:0.918045
[17]	eval-auc:0.85289	train-auc:0.922004
[18]	eval-auc:0.85766	train-auc:0.923443
[19]	eval-auc:0.857057	train-auc:0.925121
[20]	eval-auc:0.857689	train-auc:0.925506
[21]	eval-auc:0.857344	train-auc:0.925913
[22]	eval-auc:0.857804	train-auc:0.926175
[23]	eval-auc:0.858091	train-auc:0.926209
[24]	eva

In [90]:
ypred = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)

In [94]:
(sum((ypred>0.5) == y_test)) / 267.0

0.8202247191011236

In [95]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import numpy as np

In [98]:
X = np.load("./tatanic_X_train.npy")
y = np.load("./tatanic_y_train.npy")

In [101]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [102]:
import lightgbm as lgb

estimator = lgb.LGBMClassifier(num_leaves=2)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.05, 0.5, 1],
    'n_estimators': [20, 40, 60, 80, 100, 120]
}

gbm = GridSearchCV(estimator, param_grid, cv=5, scoring='roc_auc')

gbm.fit(X, y)

print('Best parameters found by grid search are:', gbm.best_params_)

Best parameters found by grid search are: {'learning_rate': 0.5, 'n_estimators': 80}


In [103]:
sum(gbm.best_estimator_.predict(X_test) == y_test) / (len(y_test)*1.0)

0.846441947565543

In [108]:
sum(
    (gbm.best_estimator_.predict_proba(X_test)[:,1]> 0.51 ) == y_test) / (len(y_test)*1.0)

0.8426966292134831

In [109]:

from xgboost.sklearn import XGBClassifier
estimator = XGBClassifier()

In [110]:
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [6],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [5], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}

In [111]:

clf = GridSearchCV(estimator, parameters, n_jobs=5, 
                   cv=5, 
                   scoring='roc_auc',
                   verbose=2, refit=True)

clf.fit(X, y)
print('Best parameters found by grid search are:', gbm.best_params_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    2.1s remaining:    3.1s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    4.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    4.0s finished


Best parameters found by grid search are: {'learning_rate': 0.5, 'n_estimators': 80}


In [112]:
sum(clf.best_estimator_.predict(X_test) == y_test) / (len(y_test)*1.0)

0.8352059925093633

In [113]:
sum(
    (clf.best_estimator_.predict_proba(X_test)[:,1]> 0.51 ) == y_test) / (len(y_test)*1.0)

0.8202247191011236